# 构建一个模块的层级包

```
graphics/
    __init__.py
    primitive/
        __init__.py
        line.py
        fill.py
        text.py
    formats/
        __init__.py
        png.py
        jpg.py
```

 文件`__init__.py`的目的是要包含不同运行级别的包的可选的初始化代码。

# 控制模块被全部导入的内容

尽管强烈反对使用 `from module import *`, 但是在定义了大量变量名的模块中频繁使用。 如果你不做任何事, 这样的导入将会导入所有不以下划线开头的。 另一方面,如果定义了` __all__` , 那么只有被列举出的东西会被导出。

如果你将 `__all__` 定义成一个空列表, 没有东西将被导入。 如果 `__all__` 包含未定义的名字, 在导入时引起`AttributeError`。

In [1]:
# somemodule.py
def spam():
    pass

def grok():
    pass

blah = 42
# Only export 'spam' and 'grok'
__all__ = ['spam', 'grok']

# 使用相对路径名导入包中子模块

```
mypackage/
    __init__.py
    A/
        __init__.py
        spam.py
        grok.py
    B/
        __init__.py
        bar.py
```

如果模块`mypackage.A.spam`要导入同目录下的模块`grok`，它应该包括的`import`语句如下：
```python
# mypackage/A/spam.py
from . import grok
```

如果模块`mypackage.A.spam`要导入不同目录下的模块`B.bar`，它应该使用的`import`语句如下：
```python
# mypackage/A/spam.py
from ..B import bar
```

`import`语句的 `.` 和 `..` 看起来很滑稽, 但它指定目录名`.`为当前目录，`..B`为目录`../B`。这种语法只适用于`from`。
```python
from . import grok # OK
import .grok # ERROR
```

相对导入只适用于在合适的包中的模块。尤其是在顶层的脚本的简单模块中，它们将不起作用。如果包的部分被作为脚本直接执行，那它们将不起作用

使用Python的-m选项来执行先前的脚本，相对导入将会正确运行

# 将模块分割成多个文件

```python
# mymodule.py
class A:
    def spam(self):
        print('A.spam')

class B(A):
    def bar(self):
        print('B.bar')
```

```
mymodule/
    __init__.py
    a.py
    b.py
```

```python
# a.py
class A:
    def spam(self):
        print('A.spam')
```

```python
# b.py
from .a import A
class B(A):
    def bar(self):
        print('B.bar')
```

```python
# __init__.py
from .a import A
from .b import B
```